In [ ]:
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as K

import numpy as np
import matplotlib.pyplot as plt
import keras
import pickle
import pandas as pd
from keras import layers
from tensorflow import data
from tensorflow.python.util.tf_export import tf_export
from keras.utils import to_categorical
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# open the serialized schema for the binary data input file
with open("../data/feature_dict.pkl", "rb") as f:
    peg_header_dict = pickle.load(f)
dictionary_keys = list(peg_header_dict.keys())
df_tfrecord_headers = pd.DataFrame({'tf_record_header':dictionary_keys})
#df_tfrecord_headers
with open("../annovar/snp_rows.pkl", "rb") as f:
    snplist = pickle.load(f)
print('snplist',snplist)
with open("../annovar/snp_cols.pkl", "rb") as f:
    snpfeatures = pickle.load(f)
print('snpfeatures',snpfeatures)
with open("../annovar/snp_mask.pkl", "rb") as f:
    snpmask = pickle.load(f)
print('snpmask',snpmask)
with open("../annovar/cpg_rows.pkl", "rb") as f:
    cpglist = pickle.load(f)
print('cpglist',cpglist)
with open("../annovar/cpg_cols.pkl", "rb") as f:
    cpgfeatures = pickle.load(f)
print('cpgfeatures',cpgfeatures)
with open("../annovar/cpg_mask.pkl", "rb") as f:
    cpgmask = pickle.load(f)
print('cpgmask',cpgmask)
#cpglist

In [ ]:
#df_gwas_pvalues = pd.read_csv('../inputs/gwas_pvalues.txt',header=None,names=['snp_id','p_value'],sep='\t')
#df_gwas_pvalues
#df_merged1 = pd.merge(df_tfrecord_headers,df_gwas_pvalues,left_on='tf_record_header',right_on='snp_id')
#df_merged1
#sig_snps = df_merged1[df_merged1['p_value']<.0001]['snp_id']
#snplist = list(sig_snps.values)
print('snplist',(snplist))
#df_ewas_pvalues = pd.read_csv('../inputs/ewas_pvalues.txt',header=None,names=['cpg_id','p_value'],sep='\t')
#df_merged2 = pd.merge(df_tfrecord_headers,df_ewas_pvalues,left_on='tf_record_header',right_on='cpg_id')
#df_merged2
#sig_cpgs = df_merged2[df_merged2['p_value']<.0001]['cpg_id']
#cpglist = list(sig_cpgs.values)
print('cpglist',(cpglist))
#df_merged2.sort_values('p_value')

In [ ]:
labellist=['label']
covlist=['subject1','Female','Age','k1a','k2a','k3a','Mono','Gran','CD4T','NK','CD8_naive','CD8pCD28nCD45RAn','PlasmaBlast']
#snplist=['subject1','GSA-rs2294942_G','rs1939015_G','rs3762352_C','rs525157_A','rs72660967_T','rs6142884_A']
#cpglist=['subject1','cg25929399','cg05704942','cg10306192','cg24088508','cg20089799','cg06961873','cg00704664','cg06961873']
featurelist = covlist+snplist+cpglist
fulllist = featurelist+labellist
# print('covlist',covlist)
shuffle_buffer = 100
sample_size = 551
repeats = 10
batch_size = 50
train_batches = 4

peg_header_dict_subset = {}
for f in fulllist:
    if f in peg_header_dict:
        peg_header_dict_subset[f] = peg_header_dict[f]
        
print('dictionary for tfrecord:',peg_header_dict_subset)


In [ ]:

# Sample string labels
labels = ['cases','controls']

feature_description = peg_header_dict_subset
def parse_single(example_proto,sublist):
    # Parse the input `tf.train.Example` proto using the dictionary above.
    diction = tf.io.parse_single_example(example_proto, feature_description)
    # convert label to a float
    label = diction['label']
    intlabel = tf.constant(0.0,dtype='float32')
    if label=='cases':
        intlabel = tf.constant(1.0,dtype='float32')
    diction['label'] = intlabel
    filteredlist = {key:diction[key] for key in sublist}
    return filteredlist

def floatcaster(features):
    casted = [tf.cast(features[column], tf.float32)  for column in features if tf.debugging.is_numeric_tensor(features[column])]
    #print('casted',casted)
    features = tf.concat(  tf.transpose(casted), 1)    
    return features
  
def parse_batched(features,covlist,snplist,cpglist,labellist):
    feature_dict = {}
    feature_dict['cov_input'] = floatcaster( {key:features[key] for key in covlist} )
    feature_dict['snp_input'] = floatcaster( {key:features[key] for key in snplist} )
    feature_dict['cpg_input'] = floatcaster( {key:features[key] for key in cpglist} )
    labels = features[labellist[0]]
    return (feature_dict,labels)
    



In [ ]:
filenames = ['../data/dataset.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filenames)
# shuffled_dataset = raw_dataset.shuffle(shuffle_buffer)
# train,val = keras.utils.split_dataset(shuffled_dataset,left_size = 0.7)
# print('train',train.cardinality())
# print('val',val.cardinality())

full_dataset = raw_dataset.map(lambda x: parse_single(x,featurelist+labellist))
print('full',full_dataset)

batched_dataset = full_dataset.repeat().shuffle(shuffle_buffer).batch(batch_size).map(lambda features:parse_batched(features,covlist,snplist,cpglist,labellist))
print('batched',batched_dataset)
#print('batched2',next(iter(batched_dataset2)))
# train,val = keras.utils.split_dataset(batched_dataset,left_size = 0.7)
train = batched_dataset.take(train_batches)
print('train',train)
val = batched_dataset.skip(train_batches)
print('val',val)
# print('cov dim',train.element_spec[0]['cov_input'].shape[1])
# print('snp dim',train.element_spec[0]['snp_input'].shape[1])
# print('cpg dim',train.element_spec[0]['cpg_input'].shape[1])
#print('training sample',next(iter(train)))

In [ ]:
@tf_export('keras.layers.MaskedDense')
class MaskedDense(layers.Layer):
    def __init__(self, units, mask, **kwargs):
        super(MaskedDense, self).__init__(**kwargs)
        self.units = units
        self.mask = mask  # Mask should be a numpy array or a tensor

    def build(self, input_shape):
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        super(MaskedDense, self).build(input_shape)

    def call(self, inputs):
        masked_kernel = self.kernel * tf.cast(self.mask, self.kernel.dtype)
        output = tf.matmul(inputs, masked_kernel)
        output = tf.nn.bias_add(output, self.bias)
        return output

    def get_config(self):
        config = super(MaskedDense, self).get_config()
        config.update({
            'units': self.units,
            'mask': self.mask.tolist() if hasattr(self.mask, 'tolist') else self.mask,  # Handle numpy arrays or tensors
        })
        return config

    @classmethod
    def from_config(cls, config):
        mask = config.pop('mask')
        return cls(mask=mask, **config)
testmask = np.zeros((4,3))
testmask[2,2] = 1
layer = MaskedDense(units=3,mask=testmask)
print('layer',layer)
print('config',layer.get_config())

In [72]:

class make_connection(tf.keras.constraints.Constraint):    
    def __init__(self, mask):
        self.mask = mask
        
    def __call__(self, w):
        # tf.keras.backend.set_value(w[0],tf.constant(1,dtype=tf.float32))
        # print('mask',self.mask)
        # print('random',w)        
        return tf.math.multiply(w,self.mask)
        # print('mult',mult)
        # tf.keras.backend.set_value(w,mult)
        # tf.keras.backend.set_value(w[1, 2], 1)
        # tf.keras.backend.set_value(w[2, 1], 1)
        # tf.keras.backend.set_value(w[2, 2], 1)    
        # return w
        
def create_cov_model(dataset):
    covariates = dataset.element_spec[0]['cov_input'].shape[1]
    print('covariates',covariates)    
    inputs = layers.Input(shape=(covariates,),name='cov_input')
    x = layers.Dense(1,name='cov_output')(inputs)
    x = layers.Activation("relu",name='cov_activation1')(x)
    return keras.models.Model(inputs,x,name='cov_model')
cov_model =create_cov_model(train)
print(cov_model.summary())
def get_snp_mask(snps,genes):
    # mask = tf.Variable(np.zeros((snps,genes),dtype='float32'))    
    
    # for gene in range(genes):
    #     for snp in range(3*gene,3*gene+3):
    #         tf.keras.backend.set_value(mask[snp,gene],1.0)
    #         #tf.keras.backend.set_value(mask[snp+1,gene],1.0)
    mask = tf.Variable(snpmask)
    print('snpmask',mask)
    return mask
def create_snp_model(dataset):
    snps = dataset.element_spec[0]['snp_input'].shape[1]            
    genes = len(snpfeatures)
    print('snps:',snps,'genes:',genes)
    snp_mask = get_snp_mask(snps,genes)    
    inputs = layers.Input(shape=(snps,),name='snp_input')
    x = layers.Dense(genes,name='snpgene_linear',kernel_constraint=make_connection(snp_mask))(inputs)
    x = layers.Activation("relu",name='snpgene_activation')(x)
    x = layers.BatchNormalization(center=False,scale=False,name='snpgene_normalization')(x)
    x = layers.Dense(1,name='snpgene_output_linear')(x)
    x = layers.Activation("relu",name='snpgene_output_relu')(x)
    return keras.models.Model(inputs,x,name='snp_model')
snp_model = create_snp_model(train)
print(snp_model.summary())
def get_cpg_mask(cpgs,genes):
    # mask = tf.Variable(np.zeros((cpgs,genes),dtype='float32'))    
    # for gene in range(genes):
    #     for cpg in range(2*gene,2*gene+2):
    #         tf.keras.backend.set_value(mask[cpg,gene],1.0)
    #         #tf.keras.backend.set_value(mask[cpg+1,gene],1.0)
    mask = tf.Variable(cpgmask)
    print('cpgmask',mask)
    return mask
def create_cpg_model(dataset):
    cpgs = dataset.element_spec[0]['cpg_input'].shape[1]
    genes = len(cpgfeatures)
    print('cpgs:',cpgs,'genes:',genes)    
    cpg_mask = get_cpg_mask(cpgs,genes)
    inputs = layers.Input(shape=(cpgs,),name='cpg_input')
    x = layers.Dense(genes,name='cpggene_linear',kernel_constraint=make_connection(cpg_mask))(inputs)
    x = layers.Activation("relu",name='cpggene_relu')(x)
    x = layers.BatchNormalization(center=False,scale=False,name='cpggene_normalization')(x)
    x = layers.Dense(1,name='cpggene_output_linear')(x)
    x = layers.Activation("relu",name='cpggene_output_relu')(x)
    return keras.models.Model(inputs,x,name='cpg_model')
cpg_model = create_cpg_model(train)
print(cpg_model.summary())

def create_pathway_model(cov_model,snp_model,cpg_model):
    pathwayInput = layers.concatenate([snp_model.output,cpg_model.output],name='combine_snp_cpg')
    print('pathwayinputshape',pathwayInput.shape)
    x = layers.BatchNormalization(center=False,scale=False,name='pathwayinput_normalization')(pathwayInput)
    x = layers.Dense(4,name='pathway_level1_linear')(x)
    x = layers.Activation("relu",name='pathway_leve11_activation')(x)
    x = layers.BatchNormalization(center=False,scale=False,name='pathway_level1_normalization')(x)
    x = layers.Dense(3,name='pathway_level_2_linear')(x)
    x = layers.Activation("relu",name='pathway_level2_activation')(x)
    x = layers.BatchNormalization(center=False,scale=False,name='pathway_level2_normalization')(x)
    x = layers.concatenate([cov_model.output,x],name='combine_cov_pathway')
    x = layers.Dense(1,name='pathway_output_linear')(x)
    x = layers.Activation("sigmoid",name='pathway_output_activation')(x)
    #return keras.models.Model(inputs=[cov_model.input,snp_model.input,cpg_model.input],outputs=x)
    return keras.models.Model(inputs={'cov_input':cov_model.input,'snp_input':snp_model.input,'cpg_input':cpg_model.input},outputs=x,name='pathway_model')
pathway_model = create_pathway_model(cov_model,snp_model,cpg_model)

print('Pathway model inputs:',pathway_model.inputs)
print('Pathway model architecture',pathway_model.summary())

covariates 12


Model: "cov_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ cov_input (InputLayer)          │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov_output (Dense)              │ (None, 1)              │            13 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cov_activation1 (Activation)    │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ snp_input           │ (None, 4679)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpg_input           │ (None, 2217)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snpgene_linear      │ (None, 3607)      │ 16,880,760 │ snp_input[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpggene_linear      │ (None, 2280)      │  5,057,040 │ cpg_input[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snpgene_activation  │ (None, 3607)      │          0 │ snpgene_linear[0… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpggene_relu        │ (None, 2280)      │          0 │ cpggene_linear[0… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snpgene_normalizat… │ (None, 3607)      │      7,214 │ snpgene_activati… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpggene_normalizat… │ (None, 2280)      │      4,560 │ cpggene_relu[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snpgene_output_lin… │ (None, 1)         │      3,608 │ snpgene_normaliz… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpggene_output_lin… │ (None, 1)         │      2,281 │ cpggene_normaliz… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ snpgene_output_relu │ (None, 1)         │          0 │ snpgene_output_l… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cpggene_output_relu │ (None, 1)         │          0 │ cpggene_output_l… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ combine_snp_cpg     │ (None, 2)         │          0 │ snpgene_output_r… │
│ (Concatenate)       │                   │            │ cpggene_output_r… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathwayinput_norma… │ (None, 2)         │          4 │ combine_snp_cpg[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level1_lin… │ (None, 4)         │         12 │ pathwayinput_nor… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_leve11_act… │ (None, 4)         │          0 │ pathway_level1_l… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pathway_level1_nor… │ (None, 4)         │          8 │ pathway_leve11_a

 Total params: 21,955,526 (83.75 MB)

 Trainable params: 21,943,734 (83.71 MB)

 Non-trainable params: 11,792 (46.06 KB)

Pathway model architecture None


In [ ]:
#print(next(iter(cov_train)).numpy())
pathway_model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

rfrun_path = '../output/'
csv_logger = K.callbacks.CSVLogger(rfrun_path + 'log.csv', append=True, separator=';')
earlystop =K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=25, verbose=1, mode='auto')
saveBestModel = K.callbacks.ModelCheckpoint(rfrun_path + "bestweight_job.keras", monitor='val_loss',
                                          verbose=1, save_best_only=True, mode='auto')
reduce_lr = K.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)


history = pathway_model.fit(train ,validation_data=val,steps_per_epoch=10,validation_steps=10,epochs=20,verbose=1,callbacks=[earlystop, saveBestModel, csv_logger, reduce_lr])

Epoch 1/20


2025-06-11 17:56:32.172619: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:227: Filling up shuffle buffer (this may take a while): 7 of 100
2025-06-11 17:56:50.897142: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 4/10 ━━━━━━━━━━━━━━━━━━━━ 37s 6s/step - accuracy: 0.9746 - loss: 0.6895

2025-06-11 17:57:49.026892: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:227: Filling up shuffle buffer (this may take a while): 41 of 100
2025-06-11 17:57:51.353446: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.



Epoch 1: val_loss improved from inf to 19.38979, saving model to ../output/bestweight_job.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 208s 18s/step - accuracy: 0.9328 - loss: 1.8558 - val_accuracy: 0.4120 - val_loss: 19.3898 - learning_rate: 0.0010
Epoch 2/20


2025-06-11 17:59:59.800329: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:227: Filling up shuffle buffer (this may take a while): 1 of 100
2025-06-11 18:00:06.114311: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 4/10 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.9754 - loss: 0.5726

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print('cov_weights_matrix',pathway_model.get_layer('cov_output').get_weights()[0])
print('cpg_gene_weights_matrix',pathway_model.get_layer('cpggene_linear').get_weights()[0])
print('snp_gene_weights_matrix',pathway_model.get_layer('snpgene_linear').get_weights()[0])